In [253]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [254]:
df = pd.read_csv('final_elo.csv')

In [255]:
df

,Unnamed: 0,SEASON_ID,AWAY_TEAM_ABBREVIATION,GAME_ID,GAME_DATE,AWAY_STL,AWAY_BLK,AWAY_PF,AWAY_PTS,AWAY_PLUS_MINUS,HOME_TEAM_ABBREVIATION,HOME_STL,HOME_BLK,HOME_PF,HOME_PTS,HOME_PLUS_MINUS,HOME_OFF_RATING,HOME_DEF_RATING,HOME_AST_TOV,HOME_AST_RATIO,HOME_OREB_PCT,HOME_DREB_PCT,HOME_REB_PCT,HOME_TM_TOV_PCT,HOME_TS_PCT,HOME_PACE,HOME_PIE,AWAY_OFF_RATING,AWAY_DEF_RATING,AWAY_AST_TOV,AWAY_AST_RATIO,AWAY_OREB_PCT,AWAY_DREB_PCT,AWAY_REB_PCT,AWAY_TM_TOV_PCT,AWAY_TS_PCT,AWAY_E_PACE,AWAY_PACE,AWAY_PIE,HomeWin
0,0,22016,NYK,21600001,2016-10-25,6,6,22,88,-29,CLE,12,5,22,117,29,115.8,87.1,2.07,20.9,0.353,0.729,0.555,14.9,0.572,101.0,0.703,87.1,115.8,0.94,13.0,0.271,0.647,0.445,17.8,0.459,103.58,101.0,0.297,1
1,1,22016,UTA,21600002,2016-10-25,9,5,19,104,-9,POR,5,3,18,113,9,122.8,114.3,1.69,18.4,0.222,0.738,0.500,14.1,0.667,91.5,0.548,114.3,122.8,1.36,15.6,0.262,0.778,0.500,15.4,0.584,94.86,91.5,0.452,1
2,2,22016,SAS,21600003,2016-10-25,13,3,19,129,29,GSW,11,6,19,100,-29,99.0,129.0,1.50,18.1,0.208,0.528,0.376,15.8,0.538,100.5,0.400,129.0,99.0,1.79,16.8,0.472,0.792,0.624,14.0,0.589,101.68,100.5,0.600,0
3,3,22016,MIA,21600004,2016-10-26,5,7,22,108,12,ORL,5,4,15,96,-12,112.5,102.1,2.25,18.9,0.415,0.679,0.550,12.5,0.519,95.0,0.570,102.1,112.5,1.82,15.1,0.321,0.585,0.450,11.7,0.474,98.68,95.0,0.430,0
4,4,22016,DAL,21600005,2016-10-26,8,8,27,121,-9,IND,11,6,23,130,9,105.2,112.1,1.73,17.0,0.246,0.784,0.491,13.0,0.541,104.6,0.429,112.1,105.2,1.88,19.5,0.216,0.754,0.509,13.8,0.602,105.46,104.6,0.571,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7054,7054,22021,TOR,22101226,2022-04-10,10,2,17,94,-11,NYK,6,5,17,105,11,109.4,97.9,1.88,22.7,0.271,0.714,0.510,17.7,0.569,96.0,0.561,97.9,109.4,1.93,19.6,0.286,0.729,0.490,14.6,0.485,100.56,96.0,0.439,1
7055,7055,22021,MIA,22101227,2022-04-10,5,4,13,111,-14,ORL,4,3,16,125,14,112.1,123.8,2.00,15.3,0.228,0.809,0.490,10.1,0.549,100.0,0.424,123.8,112.1,3.30,23.5,0.191,0.772,0.510,9.9,0.643,101.10,100.0,0.576,1
7056,7056,22021,DET,22101228,2022-04-10,4,4,16,106,-12,PHI,13,6,23,118,12,110.4,121.6,1.30,18.3,0.373,0.698,0.521,20.8,0.553,96.5,0.409,121.6,110.4,2.27,18.6,0.302,0.627,0.479,11.3,0.601,99.94,96.5,0.591,1
7057,7057,22021,SAC,22101229,2022-04-10,9,7,18,116,7,PHX,9,7,25,109,-7,110.1,117.2,2.45,18.3,0.365,0.821,0.539,11.1,0.497,99.0,0.432,117.2,110.1,1.73,20.0,0.179,0.635,0.461,15.2,0.650,102.40,99.0,0.568,0


In [256]:
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

In [257]:
# Home and road team win probabilities implied by Elo ratings and home court adjustment 
import math
import time
def win_probs(home_elo, away_elo, home_court_advantage) :
    h = math.pow(10, home_elo/400)
    r = math.pow(10, away_elo/400)
    a = math.pow(10, home_court_advantage/400) 

    denom = r + a*h
    home_prob = a*h / denom
    away_prob = r / denom 
  
    return home_prob, away_prob

  #odds the home team will win based on elo ratings and home court advantage

def home_odds_on(home_elo, away_elo, home_court_advantage) :
    h = math.pow(10, home_elo/400)
    r = math.pow(10, away_elo/400)
    a = math.pow(10, home_court_advantage/400)
    return a*h/r

#this function determines the constant used in the elo rating, based on margin of victory and difference in elo ratings
def elo_k(MOV, elo_diff):
    k = 20
    if MOV>0:
        multiplier=(MOV+3)**(0.8)/(7.5+0.006*(elo_diff))
    else:
        multiplier=(-MOV+3)**(0.8)/(7.5+0.006*(-elo_diff))
    return k*multiplier


#updates the home and away teams elo ratings after a game 

def update_elo(home_score, away_score, home_elo, away_elo, home_court_advantage) :
    home_prob, away_prob = win_probs(home_elo, away_elo, home_court_advantage) 

    if (home_score - away_score > 0) :
        home_win = 1 
        away_win = 0 
    else :
        home_win = 0 
        away_win = 1 
  
    k = elo_k(home_score - away_score, home_elo - away_elo)

    updated_home_elo = home_elo + k * (home_win - home_prob) 
    updated_away_elo = away_elo + k * (away_win - away_prob)
    
    return updated_home_elo, updated_away_elo


#takes into account prev season elo
def get_prev_elo(team, date, season, team_stats, elo_df) :
    prev_game = team_stats[team_stats['GAME_DATE'] < game_date][(team_stats['HOME_TEAM_ABBREVIATION'] == team) | (team_stats['AWAY_TEAM_ABBREVIATION'] == team)].sort_values(by = 'GAME_DATE').tail(1).iloc[0] 

    if team == prev_game['HOME_TEAM_ABBREVIATION'] :
        elo_rating = elo_df[elo_df['GAME_ID'] == prev_game['GAME_ID']]['H_Team_Elo_After'].values[0]
    else :
        elo_rating = elo_df[elo_df['GAME_ID'] == prev_game['GAME_ID']]['A_Team_Elo_After'].values[0]
  
    if prev_game['SEASON_ID'] != season :
        return (0.75 * elo_rating) + (0.25 * 1505)
    else :
        return elo_rating

In [258]:
df.sort_values(by = 'GAME_DATE', inplace = True)
df.reset_index(inplace=True, drop = True)
elo_df = pd.DataFrame(columns=['GAME_ID', 'HOME_TEAM_ABBREVIATION', 'AWAY_TEAM_ABBREVIATION', 'H_Team_Elo_Before', 'A_Team_Elo_Before', 'H_Team_Elo_After', 'A_Team_Elo_After'])
teams_elo_df = pd.DataFrame(columns=['GAME_ID','Team', 'Elo', 'Date', 'Where_Played', 'Season']) 

for index, row in df.iterrows(): 
    game_id = row['GAME_ID']
    game_date = row['GAME_DATE']
    season = row['SEASON_ID']
    h_team, a_team = row['HOME_TEAM_ABBREVIATION'], row['AWAY_TEAM_ABBREVIATION']
    h_score, a_score = row['HOME_PTS'], row['AWAY_PTS'] 

    if (h_team not in elo_df['HOME_TEAM_ABBREVIATION'].values and h_team not in elo_df['AWAY_TEAM_ABBREVIATION'].values) :
        h_team_elo_before = 1500
    else :
        h_team_elo_before = get_prev_elo(h_team, game_date, season, df, elo_df)

    if (a_team not in elo_df['HOME_TEAM_ABBREVIATION'].values and a_team not in elo_df['AWAY_TEAM_ABBREVIATION'].values) :
        a_team_elo_before = 1500
    else :
        a_team_elo_before = get_prev_elo(a_team, game_date, season, df, elo_df)

    h_team_elo_after, a_team_elo_after = update_elo(h_score, a_score, h_team_elo_before, a_team_elo_before, 69)

    new_row = {'GAME_ID': game_id, 'HOME_TEAM_ABBREVIATION': h_team, 'AWAY_TEAM_ABBREVIATION': a_team, 'H_Team_Elo_Before': h_team_elo_before, 'A_Team_Elo_Before': a_team_elo_before, \
                                                                        'H_Team_Elo_After' : h_team_elo_after, 'A_Team_Elo_After': a_team_elo_after}
    teams_row_one = {'GAME_ID': game_id,'Team': h_team, 'Elo': h_team_elo_before, 'Date': game_date, 'Where_Played': 'Home', 'Season': season}
    teams_row_two = {'GAME_ID': game_id,'Team': a_team, 'Elo': a_team_elo_before, 'Date': game_date, 'Where_Played': 'Away', 'Season': season}
  
    elo_df = elo_df.append(new_row, ignore_index = True)
    teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
    teams_elo_df = teams_elo_df.append(teams_row_two, ignore_index=True)

In [259]:
dates = list(set([d.strftime("%m-%d-%Y") for d in teams_elo_df["Date"]]))
dates = sorted(dates, key=lambda x: time.strptime(x, '%m-%d-%Y'))
teams = df["AWAY_TEAM_ABBREVIATION"]
dataset = pd.DataFrame(columns=dates)
dataset["Team"] = teams.drop_duplicates()
dataset = dataset.set_index("Team")

for index, row in teams_elo_df.iterrows():
    date = row["Date"].strftime("%m-%d-%Y")
    team = row["Team"]
    elo = row["Elo"]
    dataset[date][team] = elo

teams_elo_df['Elo'] = teams_elo_df['Elo'].astype(float)
elo_df.head(100)

,GAME_ID,HOME_TEAM_ABBREVIATION,AWAY_TEAM_ABBREVIATION,H_Team_Elo_Before,A_Team_Elo_Before,H_Team_Elo_After,A_Team_Elo_After
0,21600001,CLE,NYK,1500,1500,1517.151413,1482.848587
1,21600002,POR,UTA,1500,1500,1507.825731,1492.174269
2,21600003,GSW,SAS,1500,1500,1474.484746,1525.515254
3,21600004,ORL,MIA,1500,1500,1486.082766,1513.917234
4,21600005,IND,DAL,1500,1500,1507.825731,1492.174269
...,...,...,...,...,...,...,...
95,21600096,ORL,CHI,1482.010169,1502.233382,1456.310459,1527.933092
96,21600094,HOU,WAS,1501.375048,1470.419207,1507.754061,1464.040194
97,21600093,UTA,PHI,1509.842646,1450.30799,1521.665552,1438.485084
98,21600095,IND,CHA,1498.620113,1523.086436,1479.256715,1542.449834


In [261]:
df = pd.read_csv('final_elo.csv')
df=df.drop(labels=['AWAY_E_PACE'],axis=1)
df.head(100)

,Unnamed: 0,SEASON_ID,AWAY_TEAM_ABBREVIATION,GAME_ID,GAME_DATE,AWAY_STL,AWAY_BLK,AWAY_PF,AWAY_PTS,AWAY_PLUS_MINUS,HOME_TEAM_ABBREVIATION,HOME_STL,HOME_BLK,HOME_PF,HOME_PTS,HOME_PLUS_MINUS,HOME_OFF_RATING,HOME_DEF_RATING,HOME_AST_TOV,HOME_AST_RATIO,HOME_OREB_PCT,HOME_DREB_PCT,HOME_REB_PCT,HOME_TM_TOV_PCT,HOME_TS_PCT,HOME_PACE,HOME_PIE,AWAY_OFF_RATING,AWAY_DEF_RATING,AWAY_AST_TOV,AWAY_AST_RATIO,AWAY_OREB_PCT,AWAY_DREB_PCT,AWAY_REB_PCT,AWAY_TM_TOV_PCT,AWAY_TS_PCT,AWAY_PACE,AWAY_PIE,HomeWin
0,0,22016,NYK,21600001,2016-10-25,6,6,22,88,-29,CLE,12,5,22,117,29,115.8,87.1,2.07,20.9,0.353,0.729,0.555,14.9,0.572,101.0,0.703,87.1,115.8,0.94,13.0,0.271,0.647,0.445,17.8,0.459,101.0,0.297,1
1,1,22016,UTA,21600002,2016-10-25,9,5,19,104,-9,POR,5,3,18,113,9,122.8,114.3,1.69,18.4,0.222,0.738,0.500,14.1,0.667,91.5,0.548,114.3,122.8,1.36,15.6,0.262,0.778,0.500,15.4,0.584,91.5,0.452,1
2,2,22016,SAS,21600003,2016-10-25,13,3,19,129,29,GSW,11,6,19,100,-29,99.0,129.0,1.50,18.1,0.208,0.528,0.376,15.8,0.538,100.5,0.400,129.0,99.0,1.79,16.8,0.472,0.792,0.624,14.0,0.589,100.5,0.600,0
3,3,22016,MIA,21600004,2016-10-26,5,7,22,108,12,ORL,5,4,15,96,-12,112.5,102.1,2.25,18.9,0.415,0.679,0.550,12.5,0.519,95.0,0.570,102.1,112.5,1.82,15.1,0.321,0.585,0.450,11.7,0.474,95.0,0.430,0
4,4,22016,DAL,21600005,2016-10-26,8,8,27,121,-9,IND,11,6,23,130,9,105.2,112.1,1.73,17.0,0.246,0.784,0.491,13.0,0.541,104.6,0.429,112.1,105.2,1.88,19.5,0.216,0.754,0.509,13.8,0.602,104.6,0.571,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,22016,CHI,21600096,2016-11-07,7,4,11,112,32,ORL,6,6,23,80,-32,116.7,83.3,2.89,18.9,0.346,0.827,0.587,9.4,0.546,96.0,0.664,83.3,116.7,1.87,21.4,0.173,0.654,0.413,15.6,0.456,96.0,0.336,0
96,96,22016,MIA,21600097,2016-11-07,8,7,16,85,-12,OKC,8,5,18,97,12,91.4,105.4,1.00,10.2,0.296,0.627,0.457,12.9,0.454,92.5,0.408,105.4,91.4,1.33,17.8,0.373,0.704,0.543,19.6,0.524,92.5,0.592,1
97,97,22016,LAC,21600098,2016-11-07,8,7,20,114,32,DET,5,2,19,82,-32,85.4,120.0,1.27,11.8,NaN,0.914,NaN,11.5,0.439,95.5,0.261,120.0,85.4,2.33,22.3,0.086,NaN,NaN,12.6,0.665,95.5,0.739,0
98,98,22016,GSW,21600099,2016-11-07,14,4,17,116,10,NOP,8,1,16,106,-10,113.7,106.0,2.33,24.1,0.277,NaN,NaN,14.7,0.609,101.0,0.560,106.0,113.7,1.30,18.9,NaN,0.723,NaN,20.0,0.577,101.0,0.440,0


In [262]:
def get_avg_stats_last_n_games(team, game_date, df, n):
    
    b=np.zeros((10, 16))
    prev_game_df = df[df['GAME_DATE'] <= game_date][(df['HOME_TEAM_ABBREVIATION'] == team) | (df['AWAY_TEAM_ABBREVIATION'] == team)].sort_values(by = 'GAME_DATE').tail(n)
    j,k,l=0,0,0
    for i in prev_game_df['HOME_TEAM_ABBREVIATION'].values :
        if((i == team)):
            a=prev_game_df.iloc[j,11:27].values
            #print(a)
            b[k]=a
            k=k+1
        j=j+1
    for i in prev_game_df['AWAY_TEAM_ABBREVIATION'].values :
        if((i == team)):
            b2=prev_game_df.iloc[l,5:10].values
            b3=prev_game_df.iloc[l,27:38].values
            c=np.concatenate((b2,b3))
            b[k]=c
            k=k+1
        l=l+1
    if(k<10):
        b=b[:k]
    h_df = pd.DataFrame(b,columns = ['STL', 'BLK','PF','PTS','PLUS_MINUS','OFF_RATING','DEF_RATING'
                                 ,'AST_TOV','AST_RATIO','OREB_PCT','DREB_PCT','REB_PCT','TM_TOV_PCT','TS_PCT','PACE','PIE'])
    #h_df = prev_game_df.iloc[:,10:26]
    #a_df1 = prev_game_df.iloc[:,2]
    #a_df2 = prev_game_df.iloc[:,5:9]
    #a_df3 = prev_game_df.iloc[:,27:38]
    #a_df = pd.concat([a_df1, a_df2,a_df3])
    #print(h_df)
    #df1 = pd.concat([h_df, a_df])
    #df = df[df['Team'] == team]
    #df.drop(columns = ['Team'], inplace=True)

    return h_df.mean()

STL             6.000
BLK             6.000
PF             22.000
PTS            88.000
PLUS_MINUS    -29.000
OFF_RATING     87.100
DEF_RATING    115.800
AST_TOV         0.940
AST_RATIO      13.000
OREB_PCT        0.271
DREB_PCT        0.647
REB_PCT         0.445
TM_TOV_PCT     17.800
TS_PCT          0.459
PACE          101.000
PIE             0.297
dtype: float64

In [264]:

recent_performance_df = pd.DataFrame()
for season in df['SEASON_ID'].unique() :
    season_team_stats = df[df['SEASON_ID'] == season].sort_values(by = 'GAME_DATE').reset_index(drop = True)
    season_recent_performance_df = pd.DataFrame()

    for index, row in season_team_stats.iterrows() : 
        game_id = row['GAME_ID']
        game_date = row['GAME_DATE']
        h_team = row['HOME_TEAM_ABBREVIATION']
        a_team = row['AWAY_TEAM_ABBREVIATION']

        h_team_recent_performance = get_avg_stats_last_n_games(h_team, game_date, season_team_stats, 10)
        h_team_recent_performance.index = ['H_Last_10_Avg_' + x for x in h_team_recent_performance.index]
        a_team_recent_performance = get_avg_stats_last_n_games(a_team, game_date, season_team_stats, 10)
        a_team_recent_performance.index = ['A_Last_10_Avg_' + x for x in a_team_recent_performance.index]
        new_row = pd.concat([h_team_recent_performance, a_team_recent_performance], sort=False)
        season_recent_performance_df = season_recent_performance_df.append(new_row, ignore_index=True)
        i=i+1
    recent_performance_df = pd.concat([recent_performance_df, season_recent_performance_df])

7059

In [265]:
recent_performance_df.head(100)

,H_Last_10_Avg_STL,H_Last_10_Avg_BLK,H_Last_10_Avg_PF,H_Last_10_Avg_PTS,H_Last_10_Avg_PLUS_MINUS,H_Last_10_Avg_OFF_RATING,H_Last_10_Avg_DEF_RATING,H_Last_10_Avg_AST_TOV,H_Last_10_Avg_AST_RATIO,H_Last_10_Avg_OREB_PCT,H_Last_10_Avg_DREB_PCT,H_Last_10_Avg_REB_PCT,H_Last_10_Avg_TM_TOV_PCT,H_Last_10_Avg_TS_PCT,H_Last_10_Avg_PACE,H_Last_10_Avg_PIE,A_Last_10_Avg_STL,A_Last_10_Avg_BLK,A_Last_10_Avg_PF,A_Last_10_Avg_PTS,A_Last_10_Avg_PLUS_MINUS,A_Last_10_Avg_OFF_RATING,A_Last_10_Avg_DEF_RATING,A_Last_10_Avg_AST_TOV,A_Last_10_Avg_AST_RATIO,A_Last_10_Avg_OREB_PCT,A_Last_10_Avg_DREB_PCT,A_Last_10_Avg_REB_PCT,A_Last_10_Avg_TM_TOV_PCT,A_Last_10_Avg_TS_PCT,A_Last_10_Avg_PACE,A_Last_10_Avg_PIE
0,12.000000,5.000000,22.000000,117.000000,29.000000,115.800000,87.100000,2.070000,20.900000,0.353000,0.729000,0.555000,14.900000,0.572000,101.000000,0.703000,6.000000,6.000000,22.000000,88.000000,-29.000000,87.100000,115.800000,0.940000,13.000000,0.271000,0.647000,0.445000,17.800000,0.459000,101.000000,0.297000
1,5.000000,3.000000,18.000000,113.000000,9.000000,122.800000,114.300000,1.690000,18.400000,0.222000,0.738000,0.500000,14.100000,0.667000,91.500000,0.548000,9.000000,5.000000,19.000000,104.000000,-9.000000,114.300000,122.800000,1.360000,15.600000,0.262000,0.778000,0.500000,15.400000,0.584000,91.500000,0.452000
2,11.000000,6.000000,19.000000,100.000000,-29.000000,99.000000,129.000000,1.500000,18.100000,0.208000,0.528000,0.376000,15.800000,0.538000,100.500000,0.400000,13.000000,3.000000,19.000000,129.000000,29.000000,129.000000,99.000000,1.790000,16.800000,0.472000,0.792000,0.624000,14.000000,0.589000,100.500000,0.600000
3,10.000000,4.000000,24.000000,120.000000,6.000000,112.900000,118.800000,1.930000,20.400000,0.283000,0.674000,0.478000,14.900000,0.582000,101.000000,0.503000,10.000000,1.000000,23.000000,114.000000,-6.000000,118.800000,112.900000,1.310000,15.500000,0.326000,0.717000,0.522000,15.800000,0.608000,101.000000,0.497000
4,6.000000,3.000000,29.000000,94.000000,-19.000000,96.900000,116.500000,1.110000,15.800000,0.267000,0.696000,0.484000,18.600000,0.529000,97.000000,0.383000,5.000000,5.000000,20.000000,113.000000,19.000000,116.500000,96.900000,1.710000,17.700000,0.304000,0.733000,0.516000,14.400000,0.580000,97.000000,0.617000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5.857143,4.857143,17.000000,92.857143,-9.142857,105.885714,103.457143,2.241429,17.042857,0.319200,0.674714,0.496000,11.071429,0.504714,93.071429,0.509000,6.857143,5.142857,16.571429,107.285714,6.857143,111.200000,106.457143,1.972857,19.271429,0.312167,0.667600,0.486400,14.271429,0.557000,95.642857,0.519143
96,9.428571,5.142857,21.714286,102.714286,4.857143,99.942857,98.642857,1.082857,13.414286,0.332500,0.702000,0.519333,17.285714,0.510571,99.607143,0.500000,7.000000,6.333333,21.333333,96.333333,-1.666667,99.850000,102.333333,1.281667,14.516667,0.318667,0.707000,0.509167,16.033333,0.496833,94.531667,0.463333
97,8.000000,4.285714,16.857143,97.857143,1.857143,103.714286,98.071429,1.552857,16.342857,0.277833,0.754667,0.524000,14.257143,0.533429,96.071429,0.541714,10.285714,5.142857,20.428571,104.285714,14.857143,101.157143,95.628571,1.464286,16.071429,0.216667,0.755200,0.489000,14.600000,0.536286,98.428571,0.564143
98,9.428571,6.142857,21.714286,101.142857,-7.571429,99.100000,103.214286,1.751429,17.571429,0.226000,0.750500,0.477800,14.428571,0.519143,100.827143,0.473714,10.714286,5.428571,18.714286,112.857143,3.428571,101.728571,114.742857,1.515714,18.257143,0.237167,0.673714,0.449500,16.728571,0.541286,102.714286,0.438429


In [269]:
elo_df = elo_df.reset_index(drop=True)
recent_performance_df=recent_performance_df.reset_index(drop=True)
df_elorating=pd.concat([elo_df,recent_performance_df],axis=1)

In [270]:
df_elorating

,GAME_ID,HOME_TEAM_ABBREVIATION,AWAY_TEAM_ABBREVIATION,H_Team_Elo_Before,A_Team_Elo_Before,H_Team_Elo_After,A_Team_Elo_After,H_Last_10_Avg_STL,H_Last_10_Avg_BLK,H_Last_10_Avg_PF,H_Last_10_Avg_PTS,H_Last_10_Avg_PLUS_MINUS,H_Last_10_Avg_OFF_RATING,H_Last_10_Avg_DEF_RATING,H_Last_10_Avg_AST_TOV,H_Last_10_Avg_AST_RATIO,H_Last_10_Avg_OREB_PCT,H_Last_10_Avg_DREB_PCT,H_Last_10_Avg_REB_PCT,H_Last_10_Avg_TM_TOV_PCT,H_Last_10_Avg_TS_PCT,H_Last_10_Avg_PACE,H_Last_10_Avg_PIE,A_Last_10_Avg_STL,A_Last_10_Avg_BLK,A_Last_10_Avg_PF,A_Last_10_Avg_PTS,A_Last_10_Avg_PLUS_MINUS,A_Last_10_Avg_OFF_RATING,A_Last_10_Avg_DEF_RATING,A_Last_10_Avg_AST_TOV,A_Last_10_Avg_AST_RATIO,A_Last_10_Avg_OREB_PCT,A_Last_10_Avg_DREB_PCT,A_Last_10_Avg_REB_PCT,A_Last_10_Avg_TM_TOV_PCT,A_Last_10_Avg_TS_PCT,A_Last_10_Avg_PACE,A_Last_10_Avg_PIE
0,21600001,CLE,NYK,1500,1500,1517.151413,1482.848587,12.0,5.0,22.0,117.0,29.0,115.80,87.10,2.070,20.90,0.3530,0.7290,0.5550,14.90,0.5720,101.000,0.7030,6.0,6.0,22.0,88.0,-29.0,87.10,115.80,0.940,13.00,0.2710,0.6470,0.4450,17.80,0.4590,101.000,0.2970
1,21600002,POR,UTA,1500,1500,1507.825731,1492.174269,5.0,3.0,18.0,113.0,9.0,122.80,114.30,1.690,18.40,0.2220,0.7380,0.5000,14.10,0.6670,91.500,0.5480,9.0,5.0,19.0,104.0,-9.0,114.30,122.80,1.360,15.60,0.2620,0.7780,0.5000,15.40,0.5840,91.500,0.4520
2,21600003,GSW,SAS,1500,1500,1474.484746,1525.515254,11.0,6.0,19.0,100.0,-29.0,99.00,129.00,1.500,18.10,0.2080,0.5280,0.3760,15.80,0.5380,100.500,0.4000,13.0,3.0,19.0,129.0,29.0,129.00,99.00,1.790,16.80,0.4720,0.7920,0.6240,14.00,0.5890,100.500,0.6000
3,21600004,ORL,MIA,1500,1500,1486.082766,1513.917234,10.0,4.0,24.0,120.0,6.0,112.90,118.80,1.930,20.40,0.2830,0.6740,0.4780,14.90,0.5820,101.000,0.5030,10.0,1.0,23.0,114.0,-6.0,118.80,112.90,1.310,15.50,0.3260,0.7170,0.5220,15.80,0.6080,101.000,0.4970
4,21600005,IND,DAL,1500,1500,1507.825731,1492.174269,6.0,3.0,29.0,94.0,-19.0,96.90,116.50,1.110,15.80,0.2670,0.6960,0.4840,18.60,0.5290,97.000,0.3830,5.0,5.0,20.0,113.0,19.0,116.50,96.90,1.710,17.70,0.3040,0.7330,0.5160,14.40,0.5800,97.000,0.6170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7054,22101218,CLE,MIL,1472.681743,1606.416928,1492.878453,1586.220218,5.8,5.2,18.1,111.6,-3.0,113.54,114.58,2.376,19.77,0.2282,0.7613,0.4998,13.34,0.5894,99.200,0.4965,8.0,4.2,19.3,118.0,0.7,117.11,121.31,2.515,21.62,0.2695,0.6990,0.4831,12.52,0.5907,98.800,0.4998
7055,22101217,WAS,CHA,1444.268804,1526.585956,1431.584150,1539.270610,7.9,7.1,22.0,118.4,3.7,114.20,114.08,1.625,17.26,0.2891,0.7001,0.5074,15.46,0.5940,101.129,0.5002,8.8,5.5,20.7,115.2,-12.6,117.01,126.99,2.340,20.02,0.2690,0.6346,0.4394,13.29,0.5967,99.600,0.4318
7056,22101216,BKN,IND,1545.425208,1352.683374,1548.279556,1349.829025,8.6,4.2,22.3,114.6,1.0,112.53,112.16,2.221,18.57,0.2768,0.7342,0.4944,12.07,0.5581,101.750,0.4936,7.2,4.5,16.4,110.4,-5.8,114.81,108.90,1.896,19.60,0.2551,0.7539,0.5150,14.57,0.6009,99.234,0.5304
7057,22101222,LAC,OKC,1539.604138,1340.05049,1549.281985,1330.372644,7.2,6.3,18.5,119.5,6.1,119.15,114.79,2.180,18.95,0.2825,0.7166,0.5125,12.86,0.6062,98.622,0.5336,6.6,4.6,17.4,108.3,-9.4,112.31,111.83,2.122,18.67,0.2796,0.7632,0.5154,12.98,0.5685,99.736,0.5075


In [272]:
df_elorating = df_elorating.drop(['H_Team_Elo_After', 'A_Team_Elo_After'], 1)

In [273]:
df_elorating

,GAME_ID,HOME_TEAM_ABBREVIATION,AWAY_TEAM_ABBREVIATION,H_Team_Elo_Before,A_Team_Elo_Before,H_Last_10_Avg_STL,H_Last_10_Avg_BLK,H_Last_10_Avg_PF,H_Last_10_Avg_PTS,H_Last_10_Avg_PLUS_MINUS,H_Last_10_Avg_OFF_RATING,H_Last_10_Avg_DEF_RATING,H_Last_10_Avg_AST_TOV,H_Last_10_Avg_AST_RATIO,H_Last_10_Avg_OREB_PCT,H_Last_10_Avg_DREB_PCT,H_Last_10_Avg_REB_PCT,H_Last_10_Avg_TM_TOV_PCT,H_Last_10_Avg_TS_PCT,H_Last_10_Avg_PACE,H_Last_10_Avg_PIE,A_Last_10_Avg_STL,A_Last_10_Avg_BLK,A_Last_10_Avg_PF,A_Last_10_Avg_PTS,A_Last_10_Avg_PLUS_MINUS,A_Last_10_Avg_OFF_RATING,A_Last_10_Avg_DEF_RATING,A_Last_10_Avg_AST_TOV,A_Last_10_Avg_AST_RATIO,A_Last_10_Avg_OREB_PCT,A_Last_10_Avg_DREB_PCT,A_Last_10_Avg_REB_PCT,A_Last_10_Avg_TM_TOV_PCT,A_Last_10_Avg_TS_PCT,A_Last_10_Avg_PACE,A_Last_10_Avg_PIE
0,21600001,CLE,NYK,1500,1500,12.0,5.0,22.0,117.0,29.0,115.80,87.10,2.070,20.90,0.3530,0.7290,0.5550,14.90,0.5720,101.000,0.7030,6.0,6.0,22.0,88.0,-29.0,87.10,115.80,0.940,13.00,0.2710,0.6470,0.4450,17.80,0.4590,101.000,0.2970
1,21600002,POR,UTA,1500,1500,5.0,3.0,18.0,113.0,9.0,122.80,114.30,1.690,18.40,0.2220,0.7380,0.5000,14.10,0.6670,91.500,0.5480,9.0,5.0,19.0,104.0,-9.0,114.30,122.80,1.360,15.60,0.2620,0.7780,0.5000,15.40,0.5840,91.500,0.4520
2,21600003,GSW,SAS,1500,1500,11.0,6.0,19.0,100.0,-29.0,99.00,129.00,1.500,18.10,0.2080,0.5280,0.3760,15.80,0.5380,100.500,0.4000,13.0,3.0,19.0,129.0,29.0,129.00,99.00,1.790,16.80,0.4720,0.7920,0.6240,14.00,0.5890,100.500,0.6000
3,21600004,ORL,MIA,1500,1500,10.0,4.0,24.0,120.0,6.0,112.90,118.80,1.930,20.40,0.2830,0.6740,0.4780,14.90,0.5820,101.000,0.5030,10.0,1.0,23.0,114.0,-6.0,118.80,112.90,1.310,15.50,0.3260,0.7170,0.5220,15.80,0.6080,101.000,0.4970
4,21600005,IND,DAL,1500,1500,6.0,3.0,29.0,94.0,-19.0,96.90,116.50,1.110,15.80,0.2670,0.6960,0.4840,18.60,0.5290,97.000,0.3830,5.0,5.0,20.0,113.0,19.0,116.50,96.90,1.710,17.70,0.3040,0.7330,0.5160,14.40,0.5800,97.000,0.6170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7054,22101218,CLE,MIL,1472.681743,1606.416928,5.8,5.2,18.1,111.6,-3.0,113.54,114.58,2.376,19.77,0.2282,0.7613,0.4998,13.34,0.5894,99.200,0.4965,8.0,4.2,19.3,118.0,0.7,117.11,121.31,2.515,21.62,0.2695,0.6990,0.4831,12.52,0.5907,98.800,0.4998
7055,22101217,WAS,CHA,1444.268804,1526.585956,7.9,7.1,22.0,118.4,3.7,114.20,114.08,1.625,17.26,0.2891,0.7001,0.5074,15.46,0.5940,101.129,0.5002,8.8,5.5,20.7,115.2,-12.6,117.01,126.99,2.340,20.02,0.2690,0.6346,0.4394,13.29,0.5967,99.600,0.4318
7056,22101216,BKN,IND,1545.425208,1352.683374,8.6,4.2,22.3,114.6,1.0,112.53,112.16,2.221,18.57,0.2768,0.7342,0.4944,12.07,0.5581,101.750,0.4936,7.2,4.5,16.4,110.4,-5.8,114.81,108.90,1.896,19.60,0.2551,0.7539,0.5150,14.57,0.6009,99.234,0.5304
7057,22101222,LAC,OKC,1539.604138,1340.05049,7.2,6.3,18.5,119.5,6.1,119.15,114.79,2.180,18.95,0.2825,0.7166,0.5125,12.86,0.6062,98.622,0.5336,6.6,4.6,17.4,108.3,-9.4,112.31,111.83,2.122,18.67,0.2796,0.7632,0.5154,12.98,0.5685,99.736,0.5075


In [280]:
df_elorating=df_elorating.sort_values(by=['GAME_ID'])

In [283]:
df_elorating.to_csv('recent_performance.csv')

In [282]:
df_elorating

,GAME_ID,HOME_TEAM_ABBREVIATION,AWAY_TEAM_ABBREVIATION,H_Team_Elo_Before,A_Team_Elo_Before,H_Last_10_Avg_STL,H_Last_10_Avg_BLK,H_Last_10_Avg_PF,H_Last_10_Avg_PTS,H_Last_10_Avg_PLUS_MINUS,H_Last_10_Avg_OFF_RATING,H_Last_10_Avg_DEF_RATING,H_Last_10_Avg_AST_TOV,H_Last_10_Avg_AST_RATIO,H_Last_10_Avg_OREB_PCT,H_Last_10_Avg_DREB_PCT,H_Last_10_Avg_REB_PCT,H_Last_10_Avg_TM_TOV_PCT,H_Last_10_Avg_TS_PCT,H_Last_10_Avg_PACE,H_Last_10_Avg_PIE,A_Last_10_Avg_STL,A_Last_10_Avg_BLK,A_Last_10_Avg_PF,A_Last_10_Avg_PTS,A_Last_10_Avg_PLUS_MINUS,A_Last_10_Avg_OFF_RATING,A_Last_10_Avg_DEF_RATING,A_Last_10_Avg_AST_TOV,A_Last_10_Avg_AST_RATIO,A_Last_10_Avg_OREB_PCT,A_Last_10_Avg_DREB_PCT,A_Last_10_Avg_REB_PCT,A_Last_10_Avg_TM_TOV_PCT,A_Last_10_Avg_TS_PCT,A_Last_10_Avg_PACE,A_Last_10_Avg_PIE
0,21600001,CLE,NYK,1500,1500,12.0,5.0,22.0,117.0,29.0,115.80,87.10,2.070,20.90,0.3530,0.7290,0.5550,14.90,0.5720,101.000,0.7030,6.0,6.0,22.0,88.0,-29.0,87.10,115.80,0.940,13.00,0.2710,0.6470,0.4450,17.80,0.4590,101.000,0.2970
1,21600002,POR,UTA,1500,1500,5.0,3.0,18.0,113.0,9.0,122.80,114.30,1.690,18.40,0.2220,0.7380,0.5000,14.10,0.6670,91.500,0.5480,9.0,5.0,19.0,104.0,-9.0,114.30,122.80,1.360,15.60,0.2620,0.7780,0.5000,15.40,0.5840,91.500,0.4520
2,21600003,GSW,SAS,1500,1500,11.0,6.0,19.0,100.0,-29.0,99.00,129.00,1.500,18.10,0.2080,0.5280,0.3760,15.80,0.5380,100.500,0.4000,13.0,3.0,19.0,129.0,29.0,129.00,99.00,1.790,16.80,0.4720,0.7920,0.6240,14.00,0.5890,100.500,0.6000
3,21600004,ORL,MIA,1500,1500,10.0,4.0,24.0,120.0,6.0,112.90,118.80,1.930,20.40,0.2830,0.6740,0.4780,14.90,0.5820,101.000,0.5030,10.0,1.0,23.0,114.0,-6.0,118.80,112.90,1.310,15.50,0.3260,0.7170,0.5220,15.80,0.6080,101.000,0.4970
4,21600005,IND,DAL,1500,1500,6.0,3.0,29.0,94.0,-19.0,96.90,116.50,1.110,15.80,0.2670,0.6960,0.4840,18.60,0.5290,97.000,0.3830,5.0,5.0,20.0,113.0,19.0,116.50,96.90,1.710,17.70,0.3040,0.7330,0.5160,14.40,0.5800,97.000,0.6170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7047,22101226,NYK,TOR,1506.607511,1610.871051,10.0,3.5,17.2,114.7,1.4,121.36,113.29,2.035,20.71,0.3147,0.7283,0.5237,15.33,0.6180,97.200,0.5290,6.6,3.8,23.4,111.7,2.9,108.26,115.68,1.683,16.69,0.2519,0.7239,0.4820,14.08,0.5589,98.600,0.4539
7046,22101227,ORL,MIA,1306.280083,1642.152228,6.8,4.8,19.4,109.5,5.2,112.15,111.93,2.106,18.26,0.2592,0.7407,0.4942,12.60,0.5663,95.550,0.4945,9.8,4.2,18.9,114.7,9.3,115.73,109.73,2.149,18.25,0.3135,0.7368,0.5265,14.38,0.5757,96.805,0.5185
7045,22101228,PHI,DET,1595.539673,1382.558218,6.5,3.9,18.6,104.0,-9.0,110.01,106.12,2.104,18.78,0.2218,0.7580,0.4936,12.32,0.5647,99.345,0.5330,7.4,2.7,20.1,114.0,3.6,117.59,115.00,2.089,19.31,0.2462,0.7184,0.4835,13.91,0.6125,96.600,0.5205
7050,22101229,PHX,SAC,1686.415379,1356.545289,8.0,4.4,17.4,116.8,8.0,116.90,110.24,2.214,19.81,0.2958,0.7375,0.5213,12.88,0.5828,99.300,0.5451,5.9,3.0,18.2,117.7,9.0,120.75,114.84,2.218,18.57,0.2406,0.7405,0.5035,12.04,0.6225,96.100,0.5450
